# Project 1: Gradient-based Algorithms and Differentiable Programming


## 1. Introduction
Consider a simple formulation of rocket landing where the rocket state $x(t)$ is represented by its distance to the ground $d(t)$ and its velocity $v(t)$, i.e., $x(t) = [d_x(t), v_x(t), d_y(t), v_y(t), \theta(t)]^T$, where $t$ specifies time. The control input of the rocket is its acceleration $a(t)$ and angular velocity $\omega(t)$. The discrete-time dynamics follows 

$$
\begin{aligned}
& d_x(t+1) = d_x(t) + v_x(t) \Delta t, \\
& v_x(t+1) = v_x(t) + a(t)sin(\theta(t)) \Delta t, \\
& d_y(t+1) = d_y(t) + v_y(t) \Delta t, \\
& v_y(t+1) = v_y(t) + a(t)cos(\theta(t)) \Delta t, \\
& \theta(t+1) = \theta(t) + \omega(t) \Delta t, \\
\end{aligned}
$$

where $\Delta t$ is a time interval. Further, let the closed-loop controller be 

$$
[a(t), \omega(t)] = f_{\theta}(x(t))
$$

where $f_{\theta}(\cdot)$ is a neural network with parameters $\theta$, which are to be determined through optimization.

For each time step, we assign a loss as a function of the control input and the state: $l(x(t),a(t),\omega(t))$. In this example, we will simply set $l(x(t),a(t),\omega(t))=0$ for all $t=1,...,T-1$, where $T$ is the final time step, and $l(x(T),a(T),\omega(T)) = ||x(T)||^2 = d_x(T)^2 + v_x(T)^2 + d_y(T)^2 + v_y(T)^2 + \theta(T)^2$. This loss encourages the rocket to reach $x(T)=0$, which are proper landing conditions.

The optimization problem is now formulated as

$$
\begin{aligned}
\min_{\theta} \quad & ||x(T)||^2 \\
\quad & d_x(t+1) = d_x(t) + v_x(t) \Delta t, \\
\quad & v_x(t+1) = v_x(t) + a(t) sin(\theta(t)) \Delta t, \\
\quad & d_y(t+1) = d_y(t) + v_y(t) \Delta t, \\
\quad & v_y(t+1) = v_y(t) + a(t) cos(\theta(t)) \Delta t, \\
\quad & \theta(t+1) = \theta(t) + \omega(t) \Delta t, \\
\quad & [a(t), \omega(t)] = f_{\theta}(x(t)), ~\forall t=1,...,T-1
\end{aligned}
$$

While this problem is constrained, it is easy to see that the objective function can be expressed as a function of $x(T-1)$, $a(T-1)$ and $\omega(T-1)$, where $x(T-1)$ as a function of $x(T-2)$, $a(T-2)$ and $\omega(T-2)$, and so on. Thus it is essentially an unconstrained problem with respect to $\theta$. 

In the following, we code this problem up with [PyTorch](https://pytorch.org/), which allows us to only build the forward pass of the loss (i.e., how we move from $x(1)$ to $x(2)$ and all the way to $x(T)$) and automatically get the gradient $\nabla_{\theta} l(x(T),a(T),\omega(T))$.

---

## 2. Sample Code

Before you start, please make sure you install the PyTorch package in Python. If you are using Pycharm, you can do so through *setting*->*Project: Your Project Name*->*Project Interpreter*->*Install (the little plus sign to the right of the window)*. 

![Image](https://github.com/dayanbatuofu/MAE-598-Design-Optimization/raw/main/project1_demo/rocket%20landing.png)

In [8]:
# overhead
import logging
import math
import random
import numpy as np
import time
import torch
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR # for learning rate
import scipy.io

logger = logging.getLogger(__name__)

# set seed for reproducability
torch.manual_seed(0)
torch.random.manual_seed(0)
torch.autograd.set_detect_anomaly(True)

In [9]:
# environment parameters
dx_ini = -0.3  # initial position at x axis
vx_ini = 0  # initial velocity at x axis
dy_ini = 6  # initial position at y axis
vy_ini = 0  # initial velocity at y axis
theta = math.pi / 3  # initial angle deviation of rocket axis from y axis

FRAME_TIME = 0.1  # time interval
GRAVITY_ACCEL = 0.12  # gravity constant
BOOST_ACCEL = 3  # thrust constant

# # the following parameters are not being used in the sample code
PLATFORM_WIDTH = 0.25  # landing platform width
PLATFORM_HEIGHT = 0.06  # landing platform height
ROTATION_ACCEL = 20  # rotation constant

In [10]:
# define system dynamics
# Notes: 
# 0. You only need to modify the "forward" function
# 1. All variables in "forward" need to be PyTorch tensors.
# 2. All math operations in "forward" has to be differentiable, e.g., default PyTorch functions.
# 3. Do not use inplace operations, e.g., x += 1. Please see the following section for an example that does not work.

class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()

    def forward(self, state, action):
        """
        action[0] = controller
        action[1] = theta

        state[0] = x
        state[1] = x_dot
        state[2] = y
        state[3] = y_dot
        state[4] = theta
        """
        # Apply gravity
        # Note: Here gravity is used to change velocity which is the second element of the state vector
        # Normally, we would do x[1] = x[1] + gravity * delta_time
        # but this is not allowed in PyTorch since it overwrites one variable (x[1]) that is part of the computational graph to be differentiated.
        # Therefore, I define a tensor dx = [0., 0., 0., -gravity * delta_time, 0.], and do x = x + dx. This is allowed... 
        delta_state_gravity = torch.tensor([0., 0., 0., -GRAVITY_ACCEL * FRAME_TIME, 0.])

        # Thrust
        # Note: Same reason as above. Need a 5-by-1 tensor.
        if state[0] <= 0:
            delta_state = BOOST_ACCEL * FRAME_TIME * torch.tensor([0., torch.sin(state[4]), 0., -torch.cos(state[4]), 0]) * action[0]
        else:
            delta_state = BOOST_ACCEL * FRAME_TIME * torch.tensor([0., -torch.sin(state[4]), 0., torch.cos(state[4]), 0]) * action[0]
        
        # Theta
        delta_state_theta = FRAME_TIME * torch.tensor([0., 0., 0., 0, -1.]) * action[1]

        # Update velocity
        state = state + delta_state + delta_state_gravity + delta_state_theta

        # Update state
        # Note: Same as above. Use operators on matrices/tensors as much as possible. Do not use element-wise operators as they are considered inplace.
        step_mat = torch.tensor([[1., FRAME_TIME, 0., 0., 0.],
                                 [0., 1., 0., 0., 0.],
                                 [0., 0., 1., FRAME_TIME, 0.],
                                 [0., 0., 0., 1., 0.],
                                 [0., 0., 0., 0., 1.]])

        state = torch.matmul(step_mat, state)

        return state

In [11]:
# a deterministic controller
# Note:
# 0. You only need to change the network architecture in "__init__"
# 1. nn.Sigmoid outputs values from 0 to 1, nn.Tanh from -1 to 1
# 2. You have all the freedom to make the network wider (by increasing "dim_hidden") or deeper (by adding more lines to nn.Sequential)
# 3. Always start with something simple

class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        # little linear network with ReLU for embeddings
        self.network = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_output),
            # You can add more layers here
            nn.Sigmoid())

    def forward(self, state):
        action = self.network(state)
        return action

In [12]:
# the simulator that rolls out x(1), x(2), ..., x(T)
# Note:
# 0. Need to change "initialize_state" to optimize the controller over a distribution of initial states
# 1. self.action_trajectory and self.state_trajectory stores the action and state trajectories along time

class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.theta_trajectory = torch.empty((1, 0))
        self.u_trajectory = torch.empty((1, 0))

    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for _ in range(T):
            action = self.controller(state)
            state = self.dynamics(state, action)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
        return self.error(state)

    @staticmethod
    def initialize_state():
        state = [dx_ini, vx_ini, dy_ini, vy_ini, theta]  # TODO: need batch of initial states
        return torch.tensor(state, requires_grad=False).float()

    def error(self, state):
        return state[0]**2 + state[1]**2 + state[2]**2 + state[3]**2 + state[4]**2

In [13]:
# set up the optimizer
# Note:
# 0. LBFGS is a good choice if you don't have a large batch size (i.e., a lot of initial states to consider simultaneously)
# 1. You can also try SGD and other momentum-based methods implemented in PyTorch
# 2. You will need to customize "visualize"
# 3. loss.backward is where the gradient is calculated (d_loss/d_variables)
# 4. self.optimizer.step(closure) is where gradient descent is done

class Optimize:

    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.LBFGS(self.parameters, lr=0.001)
        self.loss_list = []

    def step(self):
        def closure():
            loss = self.simulation(self.simulation.state)
            self.optimizer.zero_grad()
            loss.backward()
            return loss

        self.optimizer.step(closure)
        return closure()

    def train(self, epochs):
        for epoch in range(epochs):
            loss = self.step()
            self.loss_list.append(loss)
            print('[%d] loss: %.3f' % (epoch + 1, loss))
        self.visualize()

    def visualize(self):
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        dx = data[:, 0]
        dy = data[:, 2]
        vx = data[:, 1]
        vy = data[:, 3]
        theta = data[:, 4]
        plt.plot(dx, dy)
        plt.title('Position Changeable for Rocket Landing')
        plt.xlabel('Rocket X Position(m)')
        plt.ylabel('Rocket Y Position(m)')
        plt.show()

        plt.plot(list(range(self.simulation.T)), vx)
        plt.title('Velocity X Changeable for Rocket Landing')
        plt.xlabel('Time Step')
        plt.ylabel('Rocket Y Velocity(m)')
        plt.show()

        plt.plot(list(range(self.simulation.T)), vy)
        plt.title('Velocity Y Changeable for Rocket Landing')
        plt.xlabel('Time Step')
        plt.ylabel('Rocket Y Velocity(m)')
        plt.show()

        plt.plot(list(range(self.simulation.T)), theta)
        plt.title('Theta Changeable for Rocket Landing')
        plt.xlabel('Time Step')
        plt.ylabel('Rocket Theta(rad)')
        plt.show()

In [ ]:
# Now it's time to run the code!

T = 100  # time steps
dim_input = 5  # state space dimensions
dim_hidden = 6  # latent dimensions
dim_output = 2  # action space dimensions
d = Dynamics()  # define dynamics
c = Controller(dim_input, dim_hidden, dim_output)  # define controller
s = Simulation(c, d, T)  # define simulation
o = Optimize(s)  # define optimizer
o.train(120)  # solve the optimization problem

![Image](https://github.com/dayanbatuofu/MAE-598-Design-Optimization/raw/main/project1_demo/image_recording/movie_rocket_landing.gif)

## 3. A Better Problem Formulation

Here is a list of things we discussed during the class that could help to make the problem more representative of the reality:

1. More realistic definition of state and action spaces: Rocket orientation, angular velocity, etc. 
2. Better dynamical model, e.g., drag
3. Constraints in state and action spaces
4. Controller design for a distribution of initial states rather than one
5. Randomness in dynamics, sensing, etc.
6. Discontinuity in modeling so that gradient cannot be computed, e.g., mechanical failures.

In this project, please choose at least one aspect from 1 to 5 from the list to improve your problem formulation and solve the resultant problem. We will address 6 when we talk about reinforcement learning.

Here is one example of problem formulation when we consider randomness in dynamics and initial states:

$$
\begin{aligned}
\min_{\theta} \quad & \mathbb{E}_{\{w(t), u(t), x(0)\}}\left[||x(T)||^2\right] \\
\quad & d(t+1) = d(t) + v(t) + w(t) \Delta t, \\
\quad & v(t+1) = v(t) + a(t) + u(t) \Delta t, \\
\quad & a(t) = f_{\theta}(x(t)), ~\forall t=1,...,T-1 \\
\quad & x(1) \sim \Pr(x(1))
\end{aligned}
$$

Here $w(t) \sim \Pr(w(t))$ and $u(t) \sim \Pr(u(t))$ are modeled as i.i.d. noises added to the dynamics, and $\Pr(x(1))$ is the distribution of initial states. We will approximate this problem using samples from $\Pr(w(t))$, $\Pr(u(t))$, and $\Pr(x(1))$. To do so, we sample $\{w(1)^{(i)},w(2)^{(i)},...,w(T)^{(i)}\}_{i}^N$ from $\Pr(w(t))$,  $\{u(1)^{(i)},u(2)^{(i)},...,u(T)^{(i)}\}_{i}^N$ from $\Pr(u(t))$, and $\{x(0)^{(i)}\}_{i}^N$ from $\Pr(x(1))$, where $N$ is the number of samples to be considered. Then we have the following problem instead:

$$
\begin{aligned}
\min_{\theta} \quad & \frac{1}{N}\sum_{i=1}^N ||x^{(i)}(T)||^2 \\
\quad & d(t+1)^{(i)} = d(t)^{(i)} + v(t)^{(i)} + w(t)^{(i)} \Delta t, \\
\quad & v(t+1)^{(i)} = v(t)^{(i)} + a(t)^{(i)} + u(t)^{(i)} \Delta t, \\
\quad & a(t)^{(i)} = f_{\theta}(x(t)^{(i)}), ~\forall t=1,...,T-1, ~i=1,...,N \\
\end{aligned}
$$

The code up this new problem, you will fully utilize the tensor operations in PyTorch. For example, the state tensor "x" will now become a N-by-2 matrix, where the first dimension represents the number of trajectories to be optimized. 

In [ ]:
# Using Randomness



## 4. Grading

* (30%) Documentation of the problem formulation: Clearly describe the objective function, the variables, the constraints, and the assumptions involved in formulating the problem.

* (40%) Programming: Like for homeworks, please push you code to your github repo. Please comment your code so that it is useful to you in the future.

* (30%) Analysis of the results: Please document the convergence and the optimal solutions (e.g., the state trajectory if it is a control problem).

* (Bonus 20%) Formulation of a problem different from rocket landing: The PyTorch framework can also be used for other engineering problems, e.g., for structure design with nonlinear mechanical properties. You get 20 bonus points for solving your own problems that are at least at the same level of difficulty as rocket landing. 